In [211]:
from bio_files_processor import OpenFasta
from das_biotools import run_genscan, DNASequence
from custom_random_forest import RandomForestClassifierCustom

from sklearn.datasets import make_classification

In [212]:
%reload_ext autoreload
%autoreload 2

# 1. Custom Random Forest

In [213]:
X, y = make_classification(n_samples=100000)
n_job_1_random_forest = RandomForestClassifierCustom(
    max_depth=30, n_estimators=100, max_features=8, random_state=42
)
n_job_2_random_forest = RandomForestClassifierCustom(
    max_depth=30, n_estimators=100, max_features=8, random_state=42
)

In [214]:
%time n_job_1_random_forest.fit(X,y,n_jobs=1)

%time n_job_2_random_forest.fit(X,y,n_jobs=2)

CPU times: user 51.7 ms, sys: 110 ms, total: 161 ms
Wall time: 2min 32s
CPU times: user 57.4 ms, sys: 134 ms, total: 191 ms
Wall time: 1min 29s


In [215]:
%time n_jobs_1_pred = n_job_1_random_forest.predict(X, n_jobs=1)

%time n_jobs_2_pred = n_job_2_random_forest.predict(X, n_jobs=2)

CPU times: user 39.4 ms, sys: 140 ms, total: 180 ms
Wall time: 3 s
CPU times: user 65.9 ms, sys: 199 ms, total: 265 ms
Wall time: 2.28 s


In [216]:
all(n_jobs_1_pred == n_jobs_2_pred)

True